In [1]:
import torch
from PIL import Image
import pandas as pd
import numpy as np


In [3]:
from importlib.resources import path

from torch import FloatTensor, LongTensor
from traitlets import Long


class MNISTDatasetASL(torch.utils.data.Dataset):
    def __init__(self, path):
        self.X, self.Y = self.get_data_mapping(path)

    def get_data_mapping(self, path):
        dataset = pd.read_csv(path)
        Y = torch.tensor(dataset['label'].values.astype(np.int8))
        X = torch.tensor(dataset.drop('label', axis = 1).values.astype(np.int8)) 
        return (X, Y)    

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        label = self.Y[idx].type(LongTensor)
        input = self.X[idx].reshape(1, 28, 28).type(FloatTensor)
        return (input, label)


In [6]:
train_path = "data/sign_mnist_train.csv"
test_path = "data/sign_mnist_test.csv"

trainLoader = torch.utils.data.DataLoader(MNISTDatasetASL(train_path), batch_size=32)
testLoader = torch.utils.data.DataLoader(MNISTDatasetASL(train_path), batch_size=32)



## Define Network Architectyre

In [45]:
import torch.nn as nn

class ASLClassifier(nn.Module):
    class ConvBlock(nn.Module):
        def __init__(self, input_c, output_c, kernel_size):
            super().__init__()
            self.block = nn.Sequential(
                nn.Conv2d(input_c, output_c, kernel_size),
                nn.ReLU(),
                nn.MaxPool2d(2,2)
            )
        def forward(self, x):
            return self.block(x)

        
    def __init__(self):
        print("HERE")
        super().__init__()

        self.conv = torch.nn.Sequential(
            self.ConvBlock(1, 6, 3),
            self.ConvBlock(6, 16, 3)
        )
        self.mlp = torch.nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 48),
            nn.ReLU(),
            nn.Linear(48, 25)
        )

    def forward(self, x):
        
        flat_out = self.conv(x).flatten(start_dim = 1)
        return self.mlp(flat_out)

    def predict(self, x):
        return torch.argmax(self.forward(x), dim=1)

net = ASLClassifier()


HERE


In [23]:
%reload_ext tensorboard
%tensorboard --logdir docs

Reusing TensorBoard on port 6006 (pid 30463), started 0:07:11 ago. (Use '!kill 30463' to kill it.)

## Set up a Training Loop

In [47]:
import torch.utils.tensorboard as tb
from os import path

def train(model, train_loader, test_loader, lr = 0.001, epochs = 2, log_dir = 'docs'):
    
    optimizer = torch.optim.AdamW(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    global_step = 0

    train_logger = tb.SummaryWriter(path.join(log_dir, 'train'))
    valid_logger = tb.SummaryWriter(path.join(log_dir, 'valid'))

    for epoch in range(epochs):
        total_loss = 0
        for X, Y in train_loader:
            X = X.type(torch.FloatTensor)
            output = model(X)
            loss = loss_fn(output, Y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_logger.add_scalar('loss', loss, global_step=global_step)

            global_step += 1

            total_loss += loss
        
        print(f"LOSS AT EPOCH {epoch} : {total_loss}")

        total_correct = 0
        count = 0
        for X, Y in test_loader:
            predictions = model.predict(X)

            total_correct += (predictions == Y).sum()
            count += len(Y)
        print(f"Valid Accuracy AT EPOCH {epoch} : {total_correct/ count}")
        valid_logger.add_scalar('accuracy', total_correct/count, global_step=global_step)

net = ASLClassifier()
train(net, trainLoader, testLoader, lr = 0.001, epochs=3)




HERE
LOSS AT EPOCH 0 : 907.0611572265625
Valid Accuracy AT EPOCH 0 : 0.9051174521446228
LOSS AT EPOCH 1 : 207.854248046875
Valid Accuracy AT EPOCH 1 : 0.9628483057022095
LOSS AT EPOCH 2 : 108.09074401855469
Valid Accuracy AT EPOCH 2 : 0.9757421016693115


In [48]:
torch.save(net.state_dict(), "checkpoint.pth")